# A Toxic Comment Identifier Application - Main Notebook
### DSC 478 - Winter 2023
### Project Type: App Dev
### Team Members: Jeff Bocek, Xuyang Ji, Anna-Lisa Vu


### Libraries

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import cycle
import seaborn as sns
from collections import defaultdict

#from sklearn's...
from sklearn.feature_extraction.text import CountVectorizer #Convert a collection of text documents to a matrix of token counts.
from sklearn.feature_extraction.text import TfidfTransformer #Transform a count matrix to a normalized tf or tf-idf representation
from sklearn.feature_extraction.text import TfidfVectorizer #Convert a collection of raw documents to a matrix of TF-IDF features
from sklearn.model_selection import train_test_split, GridSearchCV #Cross validation gird search and training and testing chunks
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.naive_bayes import ComplementNB
from sklearn.decomposition import TruncatedSVD #Dimension Reduction (LSA)
from sklearn.preprocessing import Normalizer, LabelBinarizer 
from sklearn.ensemble import VotingClassifier #to create ensemble model
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score
from sklearn.metrics import RocCurveDisplay, confusion_matrix, classification_report

## Exploratory Analysis (Xuyang)

In [4]:
#[Link to Toxic_App_Exploratory_Data_Cleaning]

## Pre-processing of Data (Xuyang)

In [5]:
#[Link to Toxic_App_Exploratory_Data_Cleaning]

### Loading Cleaned and Processed Data

Loading subset of training data with percentage of original instances:  
1% of non-toxic comments: 1433  
10% of toxic comments: 1370 and  
100% of sever toxic comments: 1595

Loading subset of testing data with percentage of original instances:  
1% of non-toxic comments: 577  
10% of toxic comments: 572 and   
100% of sever toxic comments: 367

In [ ]:
file_object = open('clean_data1.p', 'rb')
clean_data = pickle.load(file_object)
file_object.close()
train = clean_data[0]
test = clean_data[1]

In [ ]:
#balanced training data
train_df_subset1 = train['comment_text']
train_lab_subset1 = train['toxicity_level']

#testing data
test_df_subset1 = test['comment_text']
test_lab_subset1 = test['toxicity_level']

Loading **uneven** subset of training data with percentage of original instances:  
1% of non-toxic comments: 1433  
20% of toxic comments: 2740 and  
100% of sever toxic comments: 1595

In [ ]:
file_object = open('clean_data_unbalanced.p', 'rb')
clean_data = pickle.load(file_object)
file_object.close()
train = clean_data[0]
test = clean_data[1]

In [ ]:
#balanced training data
train_df_subsetU = train['comment_text']
train_lab_subsetU = train['toxicity_level']

## Data Transformation (Lisa)

In [6]:
#[Link to Toxic_App_Transformation notebook (if needed)]

### Term Frequencies (Jeff)

Simply put, the weights in the matrix represent the frequency of the term in a specific comment. The underlying concept is the higher the term frequency for a specific term in a comment, the more important it is for that comment. We use scikit-learn’s CountVectorizer(). Tuning occurred by adjusting the ‘max_df’ and ‘min_df’ which when building the vocabulary ignore terms that have a document frequency higher/lower respectively than the given threshold.

### TF*IDF (Jeff)

Term Frequency - Inverse Document Frequency assesses how important a term is within a comment relative to our collection of comments. It does this by vectorizing and scoring a term by multiplying the term’s Term Frequency (TF; number of times the term appears in the comment over the total number of terms in the comment)  by the Inverse Document Frequency (IDF; the log of; the total number of comments over the total number of comments which contain the specific term plus one). By using this formula we don’t place added importance on the super common words which overshadow less common words which can show more meaning to the comments. We used scikit-learn’s TfidfVectorizer() to transform our preprocessed collection of comments into a TF-IDF matrix. In aberration to the basic formula referenced above TfidfVectorizer() adds a “+1” to the numerator and denominator of the IDF score to prevent zero divisions when the term is not present to better represent the IDF scores for sparse matrices. We tuned this transformation by trying two methods of normalization; ‘l1’ and ‘l2’. ‘l1’  normalized by using the sum of the absolute values of the vector elements is 1 while ‘l2’ is the Euclidean norm and the sum of squares of vector elements is 1. For ‘l2’ the cosine similarity between two vectors is their dot product. As was the case with Term Frequency tuning, tuned by adjusting the ‘max_df’ and ‘min_df’ which when building the vocabulary ignore terms that have a document frequency higher/lower respectively than the given threshold. 

### Doc2Vec (Lisa)
Doc2Vec is a natural language processing technique to transform a set of documents into a list of vectors. It is closely related to the Word2Vec technique and is considered a more generalized form of it.  To best understand Doc2Vec, it is imperative to understad Word2Vec.  In Word2Vec, the algorithm looks at the context of each word and what other words surround it.  It assigns a numerical number to the word which represents the meaning of the word given its context.  For example. if Paris and France are in the same sentence, Word2Vec will create a number for each of these words which would represent that these 2 words are related.  This is exact same example is referenced in this article: https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e  Doc2Vec on the other hand generalized this technique.  Instead of having a numeric representation for the words, with Doc2Vec you can achieve a numeric representation for a document as a whole.

We thought it would be a good exercise to use this data transformation technique in our project.  Each comment can be like a document.  To create the Doc2Vec vectors, we used the doc2Vec library package from the gensim.models library.  We ran this against the train and test data and feed that into our models.

In the attached notebook are central methods we created so that we can get a doc2vec vector for the train and test data.  Using these methods each team member was able to feed a doc2vec transformed data set into the models.  Analysis and Evaluations of our the models performed with the data set is explained in the upcoming sections.

Link to Doc2Vec Notebook: [Toxic_App_Doc2Vec.ipynb](Toxic_App_Doc2Vec.ipynb)

In [ ]:
from gensim.models import doc2vec
%run Doc2Vec.ipynb
d2v_model = doc2vec.Doc2Vec(vector_size = 1000, min_count = 5, epochs = 40)

# Balanced training set
train_vectors = get_doc2vec_vectors(d2v_model, train_df_subset1)
tokenized_test_comments = tokenize_comments(test_df_subset1)
test_vectors = infer_vectors(d2v_model, tokenized_test_comments, "")

## Unsupervised Modeling for Exploratory Analysis
### Kmeans (Xuyang)

## Supervised Modeling & Hyperparameter Tuning

### Gradiant Descent/Logistic Regression (Xuyang)

### Rocchio/Nearest Centroid (Lisa)

Explanation goes here.

### Naive Bayes (Jeff)

### KNN (Jeff)

The K Nearest Neighbors model depends on a distance or similarity function to compare how far apart instances (data points) are away from each other. In classification, for a given number of points, the majority label of these “nearest neighbors” to our point in question is assigned to our point. To tune our KNN model we examined three parameters. The first was the number of neighbors (k). A larger k should reduce the effects of noise but will make the classification boundary fuzzier.

## Evaluation of Models - Model Comparisons

In [7]:
#[Dataframe of evaluation]

### Top 3 Models with Unbalanced Dataset using Ensemble